In [94]:
import geopandas as gpd
import pandas as pd
import requests
import geojson
from shapely.geometry import shape
import datetime
import re

In [32]:
# cols=pd.read_csv(r'C:\Users\Intra24\Documents\hack_tourist\02_cdensity_1_201804.csv', skiprows=0*100000, nrows=100000, sep=';',header=0 ).columns
# df_sq=pd.DataFrame(columns=cols)

# for i in range(303,400):
#     if i==0:
#         v=0
#     else:
#         v=None
#     df=pd.read_csv(r'C:\Users\Intra24\Documents\hack_tourist\02_cdensity_1_201804.csv', skiprows=i*500000, nrows=500000, sep=';',header=v)
#     df.columns=cols
#     df_sq=pd.concat([df_sq,df[df.zid==65156]], axis=0)
#     print(df_sq.shape)

# df_sq.to_csv('sq_65156.csv')

## Для учета сезонности статистика UK

In [44]:
# https://www.gov.uk/government/statistical-data-sets/museums-and-galleries-monthly-visits
df_brit=pd.read_csv(open(r'C:\Users\Intra24\Desktop\Glasgow_uni\DA and QM\Museums_and_Galleries_monthly_visits_April_to_June.csv','r'))

In [47]:
df_brit['pc_ch']=df_brit.visits.pct_change()

In [50]:
df_brit.dtypes

museum     object
year        int64
month       int64
visits    float64
pc_ch     float64
dtype: object

In [55]:
df_br_tot=df_brit[(df_brit['museum']=='TOTAL VISITOR FIGURES')&(df_brit['visits']>0)]

In [56]:
df_br_tot.to_csv('brit_mus_stat.csv')

## Open data MSK объекты

In [87]:
def get_dict_objects(list_cat, api_key):
    url='https://apidata.mos.ru/v1/datasets/?api_key=%s'%(api_key)
    r = requests.get(url)
    if r.status_code==200:
        datasets=r.json()
        print(datasets[0].keys())
        datasets[0]['ContainsGeodata']==True
        super_big_dict=dict()
        for i in range(len(datasets)):
            idi=datasets[i]['Id'] 
            if datasets[i]['ContainsGeodata'] and datasets[i]['CategoryId'] in list_cat and datasets[i]['IsArchive']==False :
                print(idi)
                url='https://apidata.mos.ru/v1/features/%d/?api_key=%s'%(idi, api_key)
                r = requests.get(url)
                super_big_dict[idi]=r.json()
                #print(idi, super_big_dict[idi].keys())
                super_big_dict[idi]['Caption']=datasets[i]['Caption']
                super_big_dict[idi]['CategoryId']=datasets[i]['CategoryId']
                super_big_dict[idi]['DepartmentId']=datasets[i]['DepartmentId']
    else: print (r)    
    return super_big_dict

In [66]:
def get_df_attract(big_dict):
    df=pd.DataFrame()
    i=0
    for k,v in big_dict.items():
        df=pd.DataFrame(v['features'])
        df['Caption']=v['Caption']
        df['CategoryId']=v['CategoryId']
        df['DepartmentId']=v['DepartmentId']
        if i==0:
            df_fin=df.copy()
        else:
            df_fin=pd.concat([df_fin, df], axis=0)
        i+=1  
    df_fin['geometry']=df_fin['geometry'].apply(lambda x: shape(x))
    df_fin['dataset_id']=df_fin['properties'].apply(lambda x: x['DatasetId'])
    df_fin=df_fin.reset_index()
    return gpd.GeoDataFrame(df_fin, crs={'init': 'epsg:4326'}, geometry='geometry')

In [91]:
def get_add_cols(df): # add columns from atributes
    l=df['properties'].apply(lambda x: list(x['Attributes'].keys()))
    s=set()
    for v in l:
        s.update(v)
    cols=[i for i in s if 'name' in str.lower(i)  or 'hours' in str.lower(i) or 'disab' in str.lower(i) or 'paid' in str.lower(i) or 'site' in i or 'id' in i or 'time' in i]
    for name in cols:
        df[name]=df['properties'].apply(lambda x:  x['Attributes'][name] if  name in  x['Attributes'].keys() else None)
    df.drop(df.isnull().mean()[df.isnull().mean()==1].index.tolist(), axis=1, inplace=True)
    return df

In [69]:
def func_val(dict_names, col, dep_id):
    #print(dict_names[dep_id][col])
    return dict_names[dep_id][col]

In [70]:
def get_dict_names(fin_cols, df_attract):
    dic_dep_fields=dict()
    for cat in df_attract['DepartmentId'].unique():
        dic_dep_fields[cat]=get_full_column(df_attract[df_attract['DepartmentId']==cat])
    dict2=dict()
    for k, v in dic_dep_fields.items():
        dict2[k]=dict()
        for col in fin_cols:
            dict2[k][col]=get_val(col,v)
    return dict2 #dic_dep_fields

In [71]:
def get_val(fin_col, values):
    l=[i for i in values if fin_col in i]
    if len(l)==0:
        return None
    elif len(l)==1:
        return l[0]
    elif fin_col=='Name':
        l=[ i for i in l if ('Winter' not in i) and ('Organiz' not in i) and ('Balance' not in i)]
        return l[0]
    else: return l[0] 

In [72]:
def get_full_column(df):
    return df.isnull().mean()[df.isnull().mean()<=0.5].index.tolist()

In [104]:
def get_fin_cols():
    return ['geometry',  'Caption', 'CategoryId', 'DepartmentId', 'dataset_id', 'global_id','Name','WorkingHours', 'DisabilityFriendly']


Достаем часы из поля WorkHours.
Данные 2 форматов : лист или строка
    Лист  - обрабатываем функцией get_hours :
            обращаемся к элементам словаря
    Строка - обрабатываем функцией get_hours_str:
            вытаскиваем регулярками и сплитом
Общая идея :вытаскиваем часы открытия и закрытия, и выходные дни недели ( дни заменяем цифрами)

In [85]:
def get_hours(list_hours):
    l_open=[]
    l_close=[]
    close_days=[]
    if 'Hours' in list_hours[0].keys():
        key_h='Hours'
    else: 
        key_h='WorkHours'
    if 'DayOfWeek' in list_hours[0].keys():
        key_d='DayOfWeek'
    else: 
        key_d='DayWeek'
    d_num=0
    for dic in list_hours:
        try:
            l_open.append(datetime.datetime.strptime(dic[key_h].split('-')[0], '%H:%M').hour)
            l_close.append(datetime.datetime.strptime(dic[key_h].split('-')[1], '%H:%M').hour) 
        except:
            l_open.append(0)
            l_close.append(23)
            close_days.append(d_num)
        d_num+=1
    return [max(l_open), min(l_close),close_days]

In [88]:

def get_hours(list_hours):
    l_open=[]
    l_close=[]
    close_days=[]
    if 'Hours' in list_hours[0].keys():
        key_h='Hours'
    else: 
        key_h='WorkHours'
    if 'DayOfWeek' in list_hours[0].keys():
        key_d='DayOfWeek'
    else: 
        key_d='DayWeek'
    d_num=0
    for dic in list_hours:
        try:
            l_open.append(datetime.datetime.strptime(dic[key_h].split('-')[0], '%H:%M').hour)
            l_close.append(datetime.datetime.strptime(dic[key_h].split('-')[1], '%H:%M').hour) 
        except:
            l_open.append(0)
            l_close.append(23)
            close_days.append(d_num)
        d_num+=1
    return [max(l_open), min(l_close),close_days]

In [89]:
def get_hours_str(st):
    lst=re.findall('[0-9][0-9]:',st)
    #print(lst)
    if len(lst)>1:
        open_time=max([int(i[:2])for n,i in enumerate(lst,0) if n%2==0])
        close_time=min([int(i[:2])for n,i in enumerate(lst,0) if n%2==1])
    else:
        open_time=0
        close_time=23
    dict_days=dict(zip(['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье'], range(0,7)))
    s=set()
    s.update([dict_days[str.lower(str.strip(i))] for i in st.split(' ') if str.lower(str.strip(i)) in dict_days.keys()])

    s.update([dict_days[str.lower(str.strip(i))] for i in st.split(',') if str.lower(str.strip(i)) in dict_days.keys()])

    if len(list(s))==0:
        list_days=list(s)
    else:
        list_days=[i for i in range(0,7) if i not in list(s)]

    return [open_time,close_time, list_days]

In [73]:
# our consts
api_key='a2327bd3ff5aceb690bbb7b3199bfddf'
list_cat=[5,162]

In [102]:
# dict_attract=get_dict_objects(list_cat,api_key) # get in json format
df_attract=get_df_attract(dict_attract) # transform to dataframe
df_attract=get_add_cols(df_attract) # extract additional columns from attributes
fin_cols=get_fin_cols() # list of fin columns
dict_names=get_dict_names(fin_cols, df_attract) # dict of assocciations with correct names
for fc in fin_cols:
    df_attract[fc]=df_attract.apply(lambda x: None if pd.isnull(func_val(dict_names, fc, x['DepartmentId'])) else x[func_val(dict_names, fc, x['DepartmentId'])], axis=1)
geo_attract=df_attract[fin_cols]

geo_attract['w_hours']=geo_attract['WorkingHours'].apply(lambda x: get_hours(x)  if type(x)==list else ( get_hours_str(x) if type(x)==str else [0,24,[]]))
geo_attract['open_hour']=geo_attract['w_hours'].apply(lambda x: x[0])
geo_attract['close_hour']=geo_attract['w_hours'].apply(lambda x: x[1])
for i in range(0,7):
    geo_attract[str(i)]=geo_attract['w_hours'].apply(lambda x: 0 if i in x[2] else 1)

geo_attract_fin=geo_attract.drop(['WorkingHours','w_hours'], axis=1)
geo_attract_fin

C:\Users\Intra24\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Intra24\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Intra24\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,geometry,properties,type,Caption,CategoryId,DepartmentId,dataset_id,global_id,Name,DisabilityFriendly,open_hour,close_hour,0,1,2,3,4,5,6
0,MULTIPOINT (37.82087 55.75880),"{'DatasetId': 493, 'VersionNumber': 3, 'Releas...",Feature,Дома культуры и клубы,5,8,493,4.770122e+08,Культурный центр «Ивановский»,None,10,22,1,1,1,1,1,1,1
1,"MULTIPOINT (37.17137 55.98769, 37.18698 55.976...","{'DatasetId': 493, 'VersionNumber': 3, 'Releas...",Feature,Дома культуры и клубы,5,8,493,4.770122e+08,Экспериментальный центр «Творческий лицей»,None,12,18,1,1,1,1,1,1,0
2,"MULTIPOINT (37.71334 55.87074, 37.67920 55.86456)","{'DatasetId': 493, 'VersionNumber': 3, 'Releas...",Feature,Дома культуры и клубы,5,8,493,4.770123e+08,Московский многофункциональный культурный центр,None,9,21,1,1,1,1,1,1,1
3,MULTIPOINT (37.73302 55.70789),"{'DatasetId': 493, 'VersionNumber': 3, 'Releas...",Feature,Дома культуры и клубы,5,8,493,4.770123e+08,Культурный центр «Москвич»,None,9,22,1,1,1,1,1,1,1
4,MULTIPOINT (37.59502 55.59728),"{'DatasetId': 493, 'VersionNumber': 3, 'Releas...",Feature,Дома культуры и клубы,5,8,493,4.770123e+08,Дом культуры «Гармония»,None,10,19,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30981,POINT (37.36412 55.83552),"{'DatasetId': 61801, 'VersionNumber': 1, 'Rele...",Feature,"Зимние активности парков, подведомственных Деп...",162,8,61801,1.021762e+09,None,None,11,22,1,1,1,1,1,1,1
30982,POINT (37.37458 55.83265),"{'DatasetId': 61801, 'VersionNumber': 1, 'Rele...",Feature,"Зимние активности парков, подведомственных Деп...",162,8,61801,1.021762e+09,None,None,13,13,1,1,1,1,1,1,1
30983,POINT (37.54106 55.66687),"{'DatasetId': 61801, 'VersionNumber': 1, 'Rele...",Feature,"Зимние активности парков, подведомственных Деп...",162,8,61801,1.021762e+09,None,None,10,21,1,1,1,1,1,1,1
30984,POINT (37.49955 55.68593),"{'DatasetId': 61801, 'VersionNumber': 1, 'Rele...",Feature,"Зимние активности парков, подведомственных Деп...",162,8,61801,1.021762e+09,None,None,10,22,1,1,1,1,1,1,1


In [103]:
geo_attract_fin.to_file('geo_attract.json', driver='GeoJSON')